In [ ]:
import pandas as pd
import numpy as np

In [ ]:
title_akas_df = pd.read_csv('title.akas.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
episode_df = pd.read_csv('title.episode.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
basics_df = pd.read_csv('title.basics.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
ratings_df = pd.read_csv('title.ratings.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
crew_df = pd.read_csv('title.crew.tsv.gz',compression='gzip', header=0, sep='\t', quotechar='"')
principals_df = pd.read_csv('title.principals.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
name_basic_df = pd.read_csv('name.basics.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
print("reading done")


In [ ]:
# create DIRECTOR CSV
name_basic_db_df = name_basic_df[~name_basic_df['primaryProfession'].isna()]
director_db_df = name_basic_db_df[name_basic_db_df['primaryProfession'].str.contains('director')]
director_db_df = director_db_df[['nconst', 'primaryName', 'birthYear', 'deathYear']]
director_db_df = director_db_df[~(director_db_df['birthYear'] == '\\N')]
director_db_df['deathYear'] = director_db_df['deathYear'].replace('\\N', np.nan)
director_db_df.to_csv('director.csv', index=False)
director_db_df.head(60)

In [ ]:
# create Actor CSV
name_basic_db_df = name_basic_df[~name_basic_df['primaryProfession'].isna()]
actors_db_df = name_basic_db_df[(name_basic_db_df['primaryProfession'].str.contains('actor')) | (name_basic_db_df['primaryProfession'].str.contains('actress'))]
actors_db_df = actors_db_df[~(actors_db_df['birthYear'] == '\\N')]
actors_db_df = actors_db_df[['nconst', 'primaryName', 'birthYear', 'deathYear']]
actors_db_df = actors_db_df[actors_db_df['primaryName'].str.len() < 100]
actors_db_df['deathYear'] = actors_db_df['deathYear'].replace('\\N', np.nan)
actors_db_df.to_csv('actors.csv', index=False)
actors_db_df


In [ ]:
# create MOVIE CSV
movie_db_df = basics_df[basics_df['titleType'] == 'movie']
movie_db_df = pd.merge(movie_db_df, ratings_df, on='tconst', how='inner')
movie_db_df = movie_db_df.drop_duplicates()
director_df = principals_df[principals_df['category'] == 'director'][['tconst', 'nconst']]
movie_db_df = pd.merge(movie_db_df, director_df, on='tconst', how='inner')
movie_db_df.drop_duplicates(subset ="tconst",keep = 'first', inplace = True)
movie_db_df = movie_db_df.drop_duplicates()
movie_db_df = movie_db_df[~(movie_db_df['genres'] == '\\N')]
movie_db_df = movie_db_df[~(movie_db_df['runtimeMinutes'] == '\\N')]
movie_db_df = movie_db_df[~(movie_db_df['startYear'] == '\\N')]
movie_db_df = movie_db_df[['tconst', 'averageRating', 'numVotes', 'startYear', 'runtimeMinutes', 'primaryTitle', 'nconst']]
movie_db_df = movie_db_df[movie_db_df['primaryTitle'].str.len() < 100]
movie_db_df = movie_db_df[movie_db_df['nconst'].isin(director_db_df['nconst'].unique())]
movie_db_df.to_csv('movie.csv', index=False)
movie_db_df

In [ ]:
# create plays_in_movies CSV
plays_in_movie_db_df = principals_df[principals_df['tconst'].isin(movie_db_df['tconst'].unique())]
plays_in_movie_db_df = plays_in_movie_db_df[plays_in_movie_db_df['category'].isin(['actor','actress'])]
plays_in_movie_db_df = plays_in_movie_db_df[['tconst', 'nconst']]
plays_in_movie_db_df = plays_in_movie_db_df.drop_duplicates()
plays_in_movie_db_df = plays_in_movie_db_df[plays_in_movie_db_df['nconst'].isin(actors_db_df['nconst'].unique())]
plays_in_movie_db_df.to_csv('plays_in_movies.csv', index=False)
plays_in_movie_db_df

In [ ]:
# create genre_movies CSV
movie_list_db_df = basics_df[basics_df['titleType'] == 'movie']
movie_genre_db_df = movie_list_db_df.join(movie_list_db_df['genres'].str.split(',', expand=True).rename(columns={0:'Genre1', 1:'Genre2', 2:'Genre3'}))
movie_genre_db_df = movie_genre_db_df[['tconst', 'Genre1', 'Genre2', 'Genre3']]
movie_genre_db_df = movie_genre_db_df[~(movie_genre_db_df['Genre1'] == '\\N')]
pd.melt(movie_genre_db_df, id_vars='tconst', value_vars=['Genre1','Genre2','Genre3'], value_name='Genre')
movie_genre_db_df = movie_genre_db_df.melt('tconst', value_name='Genre').drop(columns='variable')
movie_genre_db_df = movie_genre_db_df[~movie_genre_db_df['Genre'].isna()]
movie_genre_db_df = movie_genre_db_df[movie_genre_db_df['tconst'].isin(movie_db_df['tconst'].unique())]
movie_genre_db_df.to_csv('genre_movies.csv', index=False)
movie_genre_db_df
